In [1]:
#encoding: utf-8
import numpy as np
import pandas as pd
import xgboost as xgb
from dateutil import parser
from datetime import datetime

data_path = "../../data/"
result_path =  "../../data/result/"
feature_path =  "../../data/feature/"
analysis_path =  "../../data/analysis/"
function_path =  "../../data/function/"

# reading data
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'evaluation_public.csv')
entbase = pd.read_csv(data_path + '1entbase.csv')
alter = pd.read_csv(data_path + '2alter.csv')
branch = pd.read_csv(data_path + '3branch.csv')
invest = pd.read_csv(data_path + '4invest.csv')
right = pd.read_csv(data_path + '5right.csv', encoding = 'utf-8')
project = pd.read_csv(data_path + '6project.csv')
lawsuit = pd.read_csv(data_path + '7lawsuit.csv')
breakfaith = pd.read_csv(data_path + '8breakfaith.csv')
recruit = pd.read_csv(data_path + '9recruit.csv')
qualification = pd.read_csv(data_path + '10qualification.csv')

C:\Users\Administrator\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [162]:
import math
def translateYearTrain(date):
    s = str(date)
    if s == "nan":
        return np.nan
    return s[0:4] + '-' + s[4:6]

In [163]:
train

,EID,TARGET,ENDDATE
0,p1,0,NaN
1,p100000,0,NaN
2,p100002,0,NaN
3,p100004,0,NaN
4,p100005,0,NaN
5,p100007,0,NaN
6,p100008,0,NaN
7,p100012,0,NaN
8,p100015,0,NaN
9,p100016,0,NaN


In [164]:
train = pd.read_csv(data_path + 'train.csv')
train['ENDDATE'] = train['ENDDATE'].apply(translateYearTrain)
train.to_csv(data_path + 'train.csv', index = False)

In [165]:
train

,EID,TARGET,ENDDATE
0,p1,0,NaN
1,p100000,0,NaN
2,p100002,0,NaN
3,p100004,0,NaN
4,p100005,0,NaN
5,p100007,0,NaN
6,p100008,0,NaN
7,p100012,0,NaN
8,p100015,0,NaN
9,p100016,0,NaN


In [166]:
entbase.columns = ['PROV', 'EID', 'RGYEAR', 'HY', 'ZCZB', 'ETYPE', 'MPNUM', 'INUM',
       'ENUM', 'FINZB', 'FSTINUM', 'TZINUM']
entbase.to_csv(data_path + '1entbase.csv', index = False, encoding = 'utf-8')

In [167]:
alter = pd.read_csv(data_path + '2alter.csv')
altbe = []#变更前
altaf = []#变更后

for index, i in enumerate(alter.ALTBE.fillna(-1)):
    if i == -1:
        altbe.append(-1)
    elif (i[:3] == '人民币') & (i[-2:] == '万元'):
        if i[3:-2] == 'null':
            altbe.append(-1)
        else:
            altbe.append(float(i[3:-2]))
    elif (i[:2] == '港币') & (i[-2:] == '万元'):
        if (alter.loc[index].ALTDATE)[:4] == '2013' or (alter.loc[index].ALTDATE)[:4] == '2014':
            altbe.append(0.79 * float(i[2:-2]))
        else:
            altbe.append(0.837 * float(i[2:-2]))
    elif (i[:2] == '美元') & (i[-2:] == '万元'):
        if (alter.loc[index].ALTDATE)[:4] == '2013':
            altbe.append(6.1932 * float(i[2:-2]))
        elif (alter.loc[index].ALTDATE)[:4] == '2014':
            altbe.append(6.1428 * float(i[2:-2]))
        else:
            altbe.append(6.2284 * float(i[2:-2]))    
    elif i[-4:] == '万人民币':
        altbe.append(float(i[:-4]))   
    elif i[-3:] == '万美元':
        if (alter.loc[index].ALTDATE)[:4] == '2013':
            altbe.append(6.1932 * float(i[:-3]))
        elif (alter.loc[index].ALTDATE)[:4] == '2014':
            altbe.append(6.1428 * float(i[:-3]))
        else:
            altbe.append(6.2284 * float(i[:-3]))   
    elif i[-7:] == '(单位：万元)':
        altbe.append(float(i[:-7])) 
    elif i[-3:] == '万港元':
        if (alter.loc[index].ALTDATE)[:4] == '2013' or (alter.loc[index].ALTDATE)[:4] == '2014':
            altbe.append(0.79 * float(i[:-3]))
        else:
            altbe.append(0.837 * float(i[:-3]))
    elif i[-2:] == '万元':
        altbe.append(float(i[:-2]))
    elif i[-1:] == '万':
        altbe.append(float(i[:-1]))   
    else:
        altbe.append(float(i))

for index, i in enumerate(alter.ALTAF.fillna(-1)):
    if i == -1:
        altaf.append(-1)
    elif (i[:3] == '人民币') & (i[-2:] == '万元'):
        if i[3:-2] == 'null':
            altaf.append(-1)
        else:
            altaf.append(float(i[3:-2]))
    elif (i[:2] == '港币') & (i[-2:] == '万元'):
        if (alter.loc[index].ALTDATE)[:4] == '2013' or (alter.loc[index].ALTDATE)[:4] == '2014':
            altaf.append(0.79 * float(i[2:-2]))
        else:
            altaf.append(0.837 * float(i[2:-2]))
    elif (i[:2] == '美元') & (i[-2:] == '万元'):
        if (alter.loc[index].ALTDATE)[:4] == '2013':
            altaf.append(6.1932 * float(i[2:-2]))
        elif (alter.loc[index].ALTDATE)[:4] == '2014':
            altaf.append(6.1428 * float(i[2:-2]))
        else:
            altaf.append(6.2284 * float(i[2:-2]))    
    elif i[-4:] == '万人民币':
        altaf.append(float(i[:-4]))   
    elif i[-3:] == '万美元':
        if (alter.loc[index].ALTDATE)[:4] == '2013':
            altaf.append(6.1932 * float(i[:-3]))
        elif (alter.loc[index].ALTDATE)[:4] == '2014':
            altaf.append(6.1428 * float(i[:-3]))
        else:
            altaf.append(6.2284 * float(i[:-3]))   
    elif i[-7:] == '(单位：万元)':
        altaf.append(float(i[:-7])) 
    elif i[-3:] == '万港元':
        if (alter.loc[index].ALTDATE)[:4] == '2013' or (alter.loc[index].ALTDATE)[:4] == '2014':
            altaf.append(0.79 * float(i[:-3]))
        else:
            altaf.append(0.837 * float(i[:-3]))
    elif i[-2:] == '万元':
        altaf.append(float(i[:-2]))
    elif i[-1:] == '万':
        altaf.append(float(i[:-1]))   
    else:
        altaf.append(float(i))

alter['ALTBE'] = altbe
alter['ALTAF'] = altaf

alter.ALTBE.replace( -1,np.nan, inplace=True)#-1用空值代替
alter.ALTAF.replace(-1,np.nan, inplace=True)

alter.columns = ['EID', 'ALTERNO', 'ALTDATE', 'ALTBE', 'ALTAF']

alter.to_csv(data_path + '2alter.csv', index = False, encoding = 'utf-8')

In [168]:
alter

,EID,ALTERNO,ALTDATE,ALTBE,ALTAF
0,p2,01,2015-01,NaN,NaN
1,p2,04,2015-01,NaN,NaN
2,p2,10,2015-01,NaN,NaN
3,p2,12,2014-08,NaN,NaN
4,p2,12,2015-01,NaN,NaN
5,p11,13,2014-07,NaN,NaN
6,p13,03,2015-05,NaN,NaN
7,p17,13,2014-04,NaN,NaN
8,p17,12,2014-04,NaN,NaN
9,p23,12,2015-08,NaN,NaN


In [169]:
branch.columns = ['EID', 'TYPECODE', 'IFHOME', 'B_REYEAR', 'B_ENDYEAR']
branch.to_csv(data_path + '3branch.csv', index = False, encoding = 'utf-8')

In [170]:
invest.columns = ['EID', 'BTEID', 'IFHOME', 'BTBL', 'BTYEAR', 'BTENDYEAR']
invest.to_csv(data_path + '4invest.csv', index = False, encoding = 'utf-8')

In [171]:
right.columns = ['EID', 'RIGHTTYPE', 'TYPECODE', 'ASKDATE', 'FBDATE']
right.to_csv(data_path + '5right.csv', index = False, encoding = 'utf-8')

In [172]:
project.columns = ['EID', 'TYPECODE', 'DJDATE', 'IFHOME']
project.to_csv(data_path + '6project.csv', index = False, encoding = 'utf-8')

In [173]:
lawsuit = pd.read_csv(data_path + '7lawsuit.csv')
import math
def translateYearLaw(date):
    s = str(date)
    if s == "nan":
        return np.nan
    return s[0:4] + '-' + s[5:7]
lawsuit.columns = ['EID', 'TYPECODE', 'LAWDATE', 'LAWAMOUNT']
lawsuit['LAWDATE'] = lawsuit['LAWDATE'].apply(translateYearLaw)
lawsuit.to_csv(data_path + '7lawsuit.csv', index = False, encoding = 'utf-8')

In [174]:
breakfaith.columns = ['EID', 'TYPECODE', 'FBDATE', 'SXENDDATE']
breakfaith.to_csv(data_path + '8breakfaith.csv', index = False, encoding = 'utf-8')

In [175]:
recruit.columns = ['EID', 'WZCODE', 'POSCODE', 'RECDATE', 'PNUM']
recruit.to_csv(data_path + '9recruit.csv', index = False, encoding = 'utf-8')

In [176]:
import math
def translateYearQua(date):
    s = str(date)
    if s == 'nan':
        return np.nan
    
    if len(date) == 7:
        return s[0:4] + '-0' + s[5]
    else:
        return s[0:4] + '-' + s[5:7]

In [177]:
qualification = pd.read_csv(data_path + '10qualification.csv')

qualification['BEGINDATE'] = qualification['BEGINDATE'].apply(translateYearQua)
qualification['EXPIRYDATE'] = qualification['EXPIRYDATE'].apply(translateYearQua)


In [183]:
qualification.columns =['EID', 'ADDTYPE', 'BEGINDATE', 'EXPIRYDATE']
qualification.to_csv(data_path + '10qualification.csv', index = False)

In [2]:
train1_6 = pd.read_csv(feature_path + 'train_v1.6.csv')
test1_6 = pd.read_csv(feature_path + 'test_v1.6.csv')

In [3]:
train1_6['len_EID_NUMBER'] = train1_6['EID_NUMBER'].apply(lambda x:int(x/15))
train1_6 = train1_6.drop(['EID_NUMBER'], axis=1)
test1_6['len_EID_NUMBER'] = test1_6['EID_NUMBER'].apply(lambda x:int(x/15))
test1_6 = test1_6.drop(['EID_NUMBER'], axis=1)

In [5]:
train1_6.to_csv(feature_path + 'train_v1.6_20.csv', index = False)
test1_6.to_csv(feature_path + 'test_v1.6_20.csv', index = False)